In [ ]:
!pip install networkx matplotlib

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt


In [ ]:
def g(p,r,a,q):
    G = nx.DiGraph()
    G.add_nodes_from(p)
    G.add_nodes_from(r)
    for x,y in a.items():
        if y: G.add_edge(x,y)
    for x,y in q.items():
        if y: G.add_edge(x,y)
    return G

def d(G):
    try:
        return True, nx.find_cycle(G)
    except:
        return False, None

def s(G,c=None,t="RAG"):
    pos = nx.spring_layout(G, seed=42)
    plt.figure(figsize=(7,5))
    nx.draw_networkx_nodes(G,pos,node_size=3000,node_color="lightblue")
    nx.draw_networkx_labels(G,pos)
    if c:
        nx.draw_networkx_edges(G,pos,edgelist=[e for e in G.edges() if e not in c],arrowstyle='-|>',arrowsize=20,edge_color='blue')
        nx.draw_networkx_edges(G,pos,edgelist=c,arrowstyle='-|>',arrowsize=20,edge_color='red',width=2)
    else:
        nx.draw_networkx_edges(G,pos,edgelist=G.edges(),arrowstyle='-|>',arrowsize=20,edge_color='blue')
    plt.title(t)
    plt.axis('off')
    plt.show()

def run_single():
    n=int(input("processes: "))
    m=int(input("resources: "))
    p=[f"P{i+1}" for i in range(n)]
    r=[f"R{i+1}" for i in range(m)]
    a={}
    for x in r:
        y=int(input(f"{x} alloc to (1-{n},0): "))
        a[x]=None if y==0 else p[y-1]
    q={}
    for x in p:
        y=int(input(f"{x} request (1-{m},0): "))
        q[x]=None if y==0 else r[y-1]
    G=g(p,r,a,q)
    f,c=d(G)
    s(G,c,"Interactive RAG")
    print("DEADLOCK!" if f else "SAFE")

def run_multi():
    cases=[
        ({"R1":"P1","R2":"P2"},{"P1":"R2","P2":"R1"}),
        ({"R1":"P1","R2":None},{"P1":"R2","P2":None}),
        ({"R1":None,"R2":None},{"P1":None,"P2":None}),
        ({"R1":"P2","R2":None},{"P1":"R1","P2":None}),
        ({"R1":"P2","R2":"P1"},{"P1":"R2","P2":"R1"})
    ]
    res=[]
    for i,(a,q) in enumerate(cases):
        p=["P1","P2"]
        r=["R1","R2"]
        G=g(p,r,a,q)
        f,c=d(G)
        res.append(0 if f else 1)
        s(G,c,f"Scenario {i+1} RAG")
    plt.figure(figsize=(7,5))
    plt.bar([f"S{i+1}" for i in range(len(res))],res,color=['green' if x==1 else 'red' for x in res])
    plt.ylabel("State (1=Safe,0=Deadlock)")
    plt.title("EDA: Safe vs Deadlock")
    plt.show()


In [ ]:
c=int(input("1-Single, 2-Multi: "))
if c==1: run_single()
else: run_multi()